재행율(recall), 정밀도(precision), F-measure  
프로젝트 제안할 때, 최종산출물 평가 지표로 활용된다.

In [1]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

In [2]:
labels=[1,0,0,1,1,1,0,1,1,1] #답
preds=[0,1,1,1,1,0,1,0,1,0] #예측

In [4]:
accuracy_score(labels, preds) #(답, 예측)

0.3

In [5]:
recall_score(labels, preds) #TP/(TP+FN) 3/(3+4)

0.42857142857142855

In [6]:
precision_score(labels, preds) #TP/(TP+FP) 3/(3+3)

0.5

In [7]:
f1_score(labels, preds) # 2*(recall*pre) / (recall+pre)

0.4615384615384615

titanic 데이터
- randomforest classifier
- decisiontree classifier

In [8]:
import pandas as pd
import numpy as np

In [10]:
train = pd.read_csv('train.csv')
train.shape
test = pd.read_csv('test.csv')
test.shape
train.head()
train.info()
train.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [13]:
#결측값 평균값으로 대체
train['Age_mean'] = train['Age']
train['Age_mean'].fillna(train['Age'].mean(),inplace=True)
test['Age_mean'] = test['Age']
test['Age_mean'].fillna(test['Age'].mean(), inplace=True)

In [15]:
#성별, 승차구, 가족사항 원핫 인코딩
train['Gender'] = (train['Sex'] == 'female')
test['Gender'] = (test['Sex'] == 'female')
train['Gender'].head()

0    False
1     True
2     True
3     True
4    False
Name: Gender, dtype: bool

In [25]:
train['Embarked'].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [27]:
train['Embarked'].isnull().sum() #2
test['Embarked'].isnull().sum() #0

2

In [29]:
train['Embarked_S'] = train['Embarked'] == 'S'
train['Embarked_C'] = train['Embarked'] == 'C'
train['Embarked_Q'] = train['Embarked'] == 'Q'

test['Embarked_S'] = test['Embarked'] == 'S'
test['Embarked_C'] = test['Embarked'] == 'C'
test['Embarked_Q'] = test['Embarked'] == 'Q'

In [30]:
train[['Embarked', 'Embarked_S', 'Embarked_C', 'Embarked_Q']].head()

,Embarked,Embarked_S,Embarked_C,Embarked_Q
0,S,True,False,False
1,C,False,True,False
2,S,True,False,False
3,S,True,False,False
4,S,True,False,False


In [33]:
train['FamilySize'].value_counts()

1     537
2     161
3     102
4      29
6      22
5      15
7      12
11      7
8       6
Name: FamilySize, dtype: int64

In [38]:
#family size는 sibsp와 parch의 합 + 1
#family size 에 따라 1이면 S, 2~4면 M, 5이상이면 L 
train['FamilySize'] = train['SibSp'] + train['Parch'] + 1
train['Family'] = train['FamilySize']
train.loc[train['FamilySize'] == 1, 'Family'] = 'S'
train.loc[(train['FamilySize'] > 1) & (train['FamilySize'] < 5), 'Family'] = 'M'
train.loc[train['FamilySize'] > 4, 'Family'] = 'L'

test['FamilySize'] = test['SibSp'] + test['Parch'] + 1
test['Family'] = test['FamilySize']
test.loc[test['FamilySize'] == 1, 'Family'] = 'S'
test.loc[(test['FamilySize'] > 1) & (test['FamilySize'] < 5), 'Family'] = 'M'
test.loc[test['FamilySize'] > 4, 'Family'] = 'L'

In [39]:
train[['Family', 'FamilySize']].head()

,Family,FamilySize
0,M,2
1,M,2
2,S,1
3,M,2
4,S,1


In [40]:
#원핫인코딩
train['Family_S'] = train['Family'] == 'S'
train['Family_M'] = train['Family'] == 'M'
train['Family_L'] = train['Family'] == 'L'

test['Family_S'] = test['Family'] == 'S'
test['Family_M'] = test['Family'] == 'M'
test['Family_L'] = test['Family'] == 'L'

train[['FamilySize', 'Family', 'Family_S', 'Family_M', 'Family_L']].head()

,FamilySize,Family,Family_S,Family_M,Family_L
0,2,M,False,True,False
1,2,M,False,True,False
2,1,S,True,False,False
3,2,M,False,True,False
4,1,S,True,False,False


In [42]:
train['Family'].value_counts()

S    537
M    292
L     62
Name: Family, dtype: int64

In [43]:
#운임료 확인
train['Fare'].describe()

count    891.000000
mean      32.204208
std       49.693429
min        0.000000
25%        7.910400
50%       14.454200
75%       31.000000
max      512.329200
Name: Fare, dtype: float64

In [50]:
#객실등급은 카테고리화
train['Pclass'].value_counts()
train['Pclass'] = train['Pclass'].astype('category')
train['Pclass'].dtypes

CategoricalDtype(categories=[1, 2, 3], ordered=False)

In [ ]:
#전처리 완료

In [ ]:
#모델링을 위한 feature 뽑아오기

In [46]:
feature_names = ['Gender', 'Age_mean', 
                 'Embarked_S', 'Embarked_C', 'Embarked_Q', 
                 'Family_S', 'Family_M', 'Family_L']

In [47]:
X_train = train[feature_names] #트레인 데이터
X_train.head()

,Gender,Age_mean,Embarked_S,Embarked_C,Embarked_Q,Family_S,Family_M,Family_L
0,False,22.0,True,False,False,False,True,False
1,True,38.0,False,True,False,False,True,False
2,True,26.0,True,False,False,True,False,False
3,True,35.0,True,False,False,False,True,False
4,False,35.0,True,False,False,True,False,False


In [48]:
y_label = train['Survived'] #답

In [49]:
X_test = test[feature_names] #테스트 데이터
X_test.head()

,Gender,Age_mean,Embarked_S,Embarked_C,Embarked_Q,Family_S,Family_M,Family_L
0,False,34.5,False,False,True,True,False,False
1,True,47.0,True,False,False,False,True,False
2,False,62.0,False,False,True,True,False,False
3,False,27.0,True,False,False,True,False,False
4,True,22.0,True,False,False,False,True,False


In [ ]:
#decisiontree로 학습

In [51]:
from sklearn.tree import DecisionTreeClassifier

#decision tree 로 모델 만들기
model = DecisionTreeClassifier(max_depth=3, random_state=2021)
model

DecisionTreeClassifier(max_depth=3, random_state=2021)

In [52]:
#모델에 트레인데이터와 정답 넣어서 학습
model.fit(X_train, y_label)

DecisionTreeClassifier(max_depth=3, random_state=2021)

In [53]:
#테스트데이터 넣어서 답 예측
prediction = model.predict(X_test)
prediction.shape

(418,)

In [57]:
test['Survived'] = prediction

submissions=test[['PassengerId', 'Survived']]
submissions.to_csv('submit.csv', index=False)

In [58]:
#randomForest로 학습

In [59]:
from sklearn.ensemble import RandomForestClassifier

In [81]:
rfModel=RandomForestClassifier(n_estimators=200, max_depth=7, random_state=11) 
rfModel.fit(X_train,y_label)
preds=rfModel.predict(X_test)

In [82]:
test['Survived']=preds
submissions=test[['PassengerId', 'Survived']]
submissions.to_csv('submit2.csv', index=False)